In [5]:
import numpy as np


In [15]:
def american_option_value_at_step(S, K, T, r, sigma, N, step, option_type):
    dt = T / N  # time step
    u = np.exp(sigma * np.sqrt(dt))  # up factor
    d = 1 / u  # down factor
    p = (np.exp(r * dt) - d) / (u - d)  # risk neutral probability

    # Initialize asset price at each node
    ST = np.zeros((N+1, N+1))
    for i in range(N+1):
        ST[i, :i+1] = S * (u ** np.arange(i, -1, -1)) * (d ** np.arange(0, i+1))

    # Initialize option values at maturity
    if option_type == 'call':
        option = np.maximum(ST - K, 0)
    else:  # put
        option = np.maximum(K - ST, 0)

    # Calculate option value at each node
    for j in range(N-1, -1, -1):
        for i in range(j+1):
            early_exercise_value = (K - ST[i, j]) if option_type == 'put' else (ST[i, j] - K)
            hold_value = (option[i, j+1] * p + option[i+1, j+1] * (1-p)) * np.exp(-r * dt)
            option[i, j] = max(early_exercise_value, hold_value)

            # If we've reached the specified step, return the option value at this point
            if j == step:
                return option[0, step]

# Example: Get option value at step 50 (halfway for N=100)
S = 100  # Stock price
K = 100  # Strike price
T = 1    # Time to maturity in years
r = 0.05 # Risk-free rate
sigma = 0.2 # Volatility
N = 100  # Number of steps
price_at_step_value = american_option_value_at_step(S, K, T, r, sigma, N, 50, 'put')
print(price_at_step_value)


100.0
